In [ ]:
import bs4 as bs
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
#Chrome
chrome_options = Options()

#opens the website
driver = webdriver.Chrome(ChromeDriverManager().install()) #gets the new version of chrome 
driver.maximize_window()

url = "https://forums.edmunds.com/discussion/2444/general/x/what-car-is-right-for-me-help-me-choose"
driver.get(url)


#function to click next 
def click_next():
    try: 
        element = driver.find_element_by_css_selector("[title*='Next Page']")
        element.click()
    except IndexError: 
        print("Next button not found")

In [ ]:
messages_list = []
users_list = []
dates_list = []
nbposts_list = []
roles_list = []

num_pages = 84

for page_count in range(1,num_pages):
    time.sleep(1)    
    page = driver.page_source
    soup = BeautifulSoup(page,'lxml')

    messages = soup.find_all('div', class_='Message userContent')
    user_ids = soup.find_all('a', class_='Username js-userCard')
    dates = soup.find_all('span', class_='MItem DateCreated')
    roles = soup.find_all('span', class_='MItem RoleTitle')
    nbposts = soup.find_all('span', class_='MItem PostCount')
    

    #appending messages 
    for i in messages:
            messages_list.append(i.get_text())

    #appending dates
    for i in dates:
            dates_list.append(i.get_text())
            
    #appending users
    for i in user_ids: 
        users_list.append(i.get_text())
        
    #appending user nb of posts
    for i in nbposts: 
        nbposts_list.append(i.get_text())
    
    #appending member role
    for i in roles: 
        roles_list.append(i.get_text())
    

    print("Done scraping", page_count, "of 84 pages.")
    click_next()
    time.sleep(1)
    
df = pd.DataFrame({"Date": dates_list, "User_Id": users_list, "Message": messages_list, "NumberOfPastPosts" : nbposts_list, "Role" : roles_list})
df.to_csv('data.csv')


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

df = pd.read_csv("data.csv")
df = df.drop(['Unnamed: 0'],axis=1)

df

In [ ]:
#cleaning
#dates
from datetime import datetime
df['Date'] = df['Date'].map(lambda x: x.lstrip('\n'))
df['Date'] = pd.to_datetime(df['Date'])

#messages
df['Message'] = df['Message'].map(lambda x: x.lstrip('\n'))
#nb posts
df['NumberOfPastPosts'] = df['NumberOfPastPosts'].map(lambda x: x.lstrip('Posts: '))

df

In [ ]:
df

In [ ]:
#defining the text processing method to eliminate punctuation and stopwords

import string

#importing stopwords from the natural language toolkit library 
from nltk.corpus import stopwords

def text_process(mess):
    nopunc =[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    final_list = []
    for word in nopunc.split():
        if word.lower() not in stopwords.words('english'):
            final_list.append(word)
    return final_list

df['Message_words'] = df['Message'].apply(text_process)

In [ ]:
df

In [ ]:
import joblib 

spam_model = joblib.load('spam_detector_model.pkl')
spam_xtrain = joblib.load('xtrain.pkl')
spam_ytrain = joblib.load('ytrain.pkl')

In [ ]:
#importing our model and methods

#we choose to work with a multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

#CountVectorizer converts a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer

#The TfidfTransformer converts a collection of documents to a matrix of TF-IDF features. 
# TFIDF is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus
from sklearn.feature_extraction.text import TfidfTransformer

#pipeline creation to accelerate sample predictions

from sklearn.pipeline import Pipeline

spam_pipeline = Pipeline([
   ( 'bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',spam_model),
])


spam_pipeline.fit(spam_xtrain,spam_ytrain)

In [ ]:
def ModerateMyPost(post):
    sample_spam_predict = spam_pipeline.predict([post])
    my_mod = {'label' : str(sample_spam_predict) }
    return my_mod

In [ ]:
pd.set_option('display.max_rows', 50)


df_notnan = df[df['Message'].notna()]


labels = []
i=0
for post in df_notnan.Message:
    sample_spam_predict = spam_pipeline.predict([post])
    labels.append(sample_spam_predict)
    print('Checked message', i, ' of', len(df_notnan.Message))
    i +=1

In [ ]:
pd.set_option('display.max_rows', 20)

df_notnan

In [ ]:
df_notnan['label'] = labels

In [ ]:
for potential_spam in df_notnan[df_notnan['label'] == "spam"]['Message']:
    print("\n", potential_spam)

In [ ]:
df_clean = df_notnan


joblib.dump(df_clean, "df_clean.pkl")